In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
import datetime
from decimal import Decimal
import pandas as pd


# Chrome options setup
chrome_options = webdriver.ChromeOptions()


# To disable notifications, geolocation, and media stream
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--disable-geolocation')
chrome_options.add_argument('--use-fake-ui-for-media-stream')
chrome_options.add_argument("--enable-features=ReaderMode")

driver=webdriver.Chrome(chrome_options)

# Maximize the browser window
driver.maximize_window()

#Passing the Webiste URL
driver.get("https://www.redbus.in/")

print("\nTitle reads : = "+driver.title)

# try:
element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,"//a[text()='View All' and @href='https://www.redbus.in/online-booking/rtc-directory']")))
view_all_element=driver.find_element(By.XPATH,"//a[text()='View All' and @href='https://www.redbus.in/online-booking/rtc-directory']")
view_all_element.click()
time.sleep(5)   

# window_handles[1] is a second window 
driver.switch_to.window(driver.window_handles[1]) 

# prints the title of the second window 
print("\nTitle reads : " + driver.title) 

# xpath of state links
list_of_states = driver.find_elements(By.XPATH, "//a[@class='D113_link']") 

counter=0
Red_bus = []
state_list = []
format_str = "%H:%M"
selected_state_list = ["KSRTC (Kerala)", "APSRTC", "TSRTC", "South Bengal State Transport Corporation (SBSTC)", "West Bengal Transport Corporation","UPSRTC", "HRTC", "Sikkim Nationalised Transport (SNT)", "Meghalaya Transport Corporation(MTC)"]

#Convert String to Time
def convert_string_to_time(local_date):
    format_str = "%H:%M"
    dt = datetime.strptime(local_date, format_str)
    t = dt.time()
    return t

#Convert Rating into float
def convert_rating(local_text):
    if local_text =="New" or local_text =="" or local_text ==" ":            
        result=0.00
    else:
        result= float(local_text.split()[0])   
    return result

# Iterate through all the state
for state in list_of_states:
    if state.text in selected_state_list:
        state_list.append(state)
        counter=counter+1
        print("{}:{}".format(counter,state.text))

for state in state_list:
    driver.implicitly_wait(10)
    state_url=state.get_attribute('href')
    local_state=""
    local_state=state.text
    driver.switch_to.new_window('tab')
    driver.implicitly_wait(10)
    driver.get(state_url)
    print("\nTraversing through {} state routes".format(local_state))

    list_of_routes= driver.find_elements (By.XPATH,"//a[@class='route']")
    
    for route in list_of_routes:
        driver.implicitly_wait(10)
        route_url=""
        route_url=route.get_attribute('href')
        local_route=""
        local_route=route.text
        driver.switch_to.new_window('tab')
        driver.implicitly_wait(10)
        driver.get(route_url)
        print("\nTraversing through {} route's buses".format(local_route))
        
        try:
            # Wait for the bus count element to be present
            bus_count = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.f-bold.busFound")))
            temp = bus_count.text 
            BusCount = int(temp.split()[0])
            print("Total Bus Count of the {} is {}".format(local_route,BusCount))

            scroll_attempts = 0    
            netCount = BusCount
            max_attempts = int(BusCount/7.5) + 2  
            
            while scroll_attempts < max_attempts:
                driver.execute_script("window.scrollBy(0, 3000);")
                scroll_attempts += 1
                
            time.sleep(3)

            # Get bus details elements       
            list_of_buses= driver.find_elements(By.XPATH,"//li[@class='row-sec clearfix']")

            for bus in list_of_buses:
                js_code3 = "arguments[0].scrollIntoView();"
                driver.execute_script(js_code3, bus)
                # Get bus details elements
                bus_name = bus.find_element(By.CSS_SELECTOR, '.travels').text
                bus_type = bus.find_element(By.CSS_SELECTOR,'.bus-type').text
                departure_time = bus.find_element(By.CSS_SELECTOR, '.dp-time').text
                arrival_time = bus.find_element(By.CSS_SELECTOR, '.bp-time').text
                price = bus.find_element(By.CSS_SELECTOR, '.fare').text
                duration = bus.find_element(By.CSS_SELECTOR, '.dur').text
                star_rating= bus.find_element(By.CSS_SELECTOR,"div[class='column-six p-right-10 w-10 fl']").text
                seats_available=bus.find_element(By.CSS_SELECTOR,'.seat-left').text
            

                #conversion
                converted_departure_time = convert_string_to_time(departure_time)
                converted_arrival_time = convert_string_to_time(arrival_time)
                converted_star_rating= convert_rating(star_rating)
                converted_seats_available=int(seats_available.split()[0])
                converted_price=Decimal(price.split()[-1])
                                

                print('''\nTraversing through {} state route {} and the URL is {}
                \nTravels:{}\tType:{} 
                \n Duration:{}\tDeparture Time:{}\tArrival Time:{}
                \n Seat Available:{}\tPrice:{}\tStar Rating:{}
                '''.format(local_state,local_route,route_url,bus_name,bus_type,duration,converted_departure_time,converted_arrival_time,converted_seats_available,converted_price,converted_star_rating))

                # Append to list
                Red_bus.append({
                    'route_name': local_route,
                    'route_link': route_url,
                    'busname': bus_name,
                    'bustype':bus_type,
                    'departing_time': converted_departure_time,
                    'duration':duration,
                    'reaching_time':converted_arrival_time,
                    'star_rating':converted_star_rating,
                    'price':converted_price,
                    'seats_available':converted_seats_available
                })

      
            driver.close()
            time.sleep(10)
            driver.switch_to.window(driver.window_handles[2])

        except TimeoutException as e:
            print("Temporarily No buses are available on this route")
            driver.close()
            time.sleep(5)
            driver.switch_to.window(driver.window_handles[2])                                   

    driver.close()
    time.sleep(10)
    driver.switch_to.window(driver.window_handles[1])

    

driver.back()
time.sleep(10)
driver.back()
driver.quit()

In [5]:
# Pushing the list into DataFrame
df = pd.DataFrame(Red_bus)

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as a string
timestamp = now.strftime("%Y%m%d_%H%M%S")

# Create the filename with the current date and time
filename = f"Red_Bus_{timestamp}.csv"

# Extract file for backup purpose
df.to_csv(filename)

In [ ]:
#Creating a new index column and adding it into Dataframe
df['ID']= range(1,len(df)+1)
df.set_index('ID',inplace=True)

# Print the DataFrame
print(df)


In [7]:
#MySQL Connection

from sqlalchemy import create_engine, text
import json

def read_config(file_path):
    with open(file_path, 'r') as file:
        config = json.load(file)
    return config

# Reading database connection parameters from Configuration file
config = read_config('config.json')

# Define database connection parameters
username = config.get('username')
password = config.get('password')
host= config.get('host')
port= config.get('port')

database = 'Red_Bus'

# Create SQLAlchemy engine
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}')


In [48]:
#Inserting DataFrame into table, created in MySQL.

# Define the table name
table_name = 'bus_routes'
 

#Truncate the table before inserting 
with engine.connect() as connection:
    sql_query = text("Truncate table bus_routes")
     # Execute the query
    result = connection.execute(sql_query)

#Insert DataFrame into MySQL table
df.to_sql(name=table_name, con=engine, if_exists='replace', index=True)
